In [ ]:
import matplotlib.pyplot as plt
import pickle

def graphGame(q):
    t = q.copy()
    t["Odds"] = t["Total_Time"].apply(lambda x: (x / 3600)) ## temp linear odds
    period_ends = t[t["Event"] == "PEND"]["Total_Time"].drop_duplicates().to_list()
    for pend in period_ends:
        plt.axvline(x=pend, color='gray', linestyle='--', alpha=0.5)

    

    for n in range(1, 10):
        plt.plot(t["Total_Time"], t["Odds"], color='red', alpha=0.1/n, linewidth=2 + n)
    plt.plot(t["Total_Time"], t["Odds"], color='black', linewidth=2)
    plt.xlabel('Seconds Elapsed')
    plt.yticks(ticks=[i/10 for i in range(0, 11)], labels=[f'{i*10}%' for i in range(0, 11)])
    plt.ylabel('Win Odds')
    plt.title('Seconds Elapsed vs Win Odds')
    plt.show()



with open("model.pkl", "rb") as file:
    model = pickle.load(file)

# Use the loaded model for predictions
predictions = model.predict(data)